# Method: VCT regression

In [1]:
#import warnings
#warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)
import numpy as np
import yaml

from vessel_manoeuvring_models.visualization.plot import track_plot, plot
from phd.visualization.plot_ship import track_plots
from vessel_manoeuvring_models.prime_system import PrimeSystem
from phd.visualization.plot_prediction import plot_total_force, plot_force_components, predict, plot_compare_model_forces, plot_parameter_contributions
from phd.visualization.bokeh_plotting import create_tab
from vessel_manoeuvring_models.symbols import *
import matplotlib.pyplot as plt
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify, remove_functions, prime
from vessel_manoeuvring_models.parameters import df_parameters
p = df_parameters["symbol"]
import statsmodels.api as sm
import sympy as sp
from sympy import Eq, symbols, latex
import paper
from paper import save_fig, save_eq
import vessel_manoeuvring_models
from phd.pipelines.regression_VCT.nodes import df_VCT_to_prime, regress_VCT
from phd.pipelines.regression_VCT.regression_pipeline import pipeline

Loading BokehJS ...

In [3]:
#%reload_kedro
ship='wPCC'
ship_data = catalog.load(f"{ship}.ship_data")
model_loaders = catalog.load(f"{ship}.models_VCT")
#models_rudder_VCT_loaders = catalog.load(f"{ship}.models_rudder_VCT")
df_VCT = catalog.load(f"{ship}.df_VCT_scaled")
exclude_parameters = catalog.load(f"params:{ship}.VCT_exclude_parameters")

[01/10/24 14:15:30] INFO     Loading data from 'wPCC.ship_data' (YAMLDataSet)...                ]8;id=797049;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=159609;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    WARNING  /home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/par ]8;id=924562;file:///home/maa/anaconda3/envs/phd/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=997561;file:///home/maa/anaconda3/envs/phd/lib/python3.10/warnings.py#109\109]8;;\
                             titioned_dataset.py:200: KedroDeprecationWarning: 'PartitionedDataset'                
                             has been moved to `kedro-datasets` and will be removed in Kedro                       
                             0.19.0.                                                                               
                               warnings.warn(                                                                      
                                                                                                                   

                    WARNING  /home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/__i ]8;id=674146;file:///home/maa/anaconda3/envs/phd/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=538410;file:///home/maa/anaconda3/envs/phd/lib/python3.10/warnings.py#109\109]8;;\
                             nit__.py:44: KedroDeprecationWarning: 'AbstractDataSet' has been                      
                             renamed to 'AbstractDataset', and the alias will be removed in Kedro                  
                             0.19.0                                                                                
                               return getattr(kedro.io.core, name)                                                 
                                                                                                                   

                    INFO     Loading data from 'wPCC.models_VCT' (PartitionedDataset)...        ]8;id=650010;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=500005;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'wPCC.df_VCT_scaled' (CSVDataSet)...             ]8;id=878748;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=830215;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'params:wPCC.VCT_exclude_parameters'             ]8;id=801377;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=570565;file:///home/maa/anaconda3/envs/phd/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\
                             (MemoryDataset)...                                                                    

## Mathematical rudder model

In [4]:
model = model_loaders['semiempirical_covered']()

In [5]:
model.show_subsystems()

hull: PrimeEquationSubSystem
propellers: PropellersSimpleSystem
rudder_port: SemiempiricalRudderSystemCovered
rudder_stbd: SemiempiricalRudderSystemCovered
rudders: Rudders
rudder_hull_interaction: RudderHullInteractionSystem
wind_force: DummyWindForceSystem


In [6]:
df_VCT_prime = df_VCT_to_prime(model=model, df_VCT=df_VCT)

In [7]:
regression_pipeline = pipeline(df_VCT_prime=df_VCT_prime, model=model)

In [8]:
regression_pipeline.keys()

dict_keys(['rudder hull interaction aH', 'rudder hull interaction xH', 'resistance', 'resistance fy', 'resistance mz', 'Drift angle fx', 'Drift angle fy', 'Drift angle mz', 'Circle fx', 'Circle fy', 'Circle mz', 'Circle + Drift fx', 'Circle + Drift fy', 'Circle + Drift mz'])

In [10]:
regression = regression_pipeline['Drift angle fy']

In [11]:
regression['eq']

Eq(Y_H, Y_{0} + Y_{vvv}*v**3 + Y_{v}*v)

In [12]:
for name,regression in regression_pipeline.items():
    display(regression['eq'])

Eq(Y_D, Y_R*a_H)

Eq(N_D, N_R*x_H)

Eq(X_H, X_{0} + X_{u}*u)

Eq(Y_H, Y_{0})

Eq(N_H, N_{0})

...

Eq(Y_H, Y_{0} + Y_{rrr}*r**3 + Y_{r}*r)

Eq(N_H, N_{0} + N_{rrr}*r**3 + N_{r}*r)

Eq(X_H, X_{0} + X_{rr}*r**2 + X_{u}*u + X_{vr}*r*v + X_{vv}*v**2)

Eq(Y_H, Y_{0} + Y_{rrr}*r**3 + Y_{r}*r + Y_{vrr}*r**2*v + Y_{vvr}*r*v**2 + Y_{vvv}*v**3 + Y_{v}*v)

Eq(N_H, N_{0} + N_{rrr}*r**3 + N_{r}*r + N_{vrr}*r**2*v + N_{vvr}*r*v**2 + N_{vvv}*v**3 + N_{v}*v)

In [13]:
df_VCT['test type'].unique()

array(['Circle', 'Circle + Drift', 'Circle + rudder angle', 'Drift angle',
       'Rudder and drift angle', 'Rudder angle', 'Thrust variation',
       'self propulsion', 'Rudder angle resistance (no propeller)'],
      dtype=object)

In [14]:
df_VCT.groupby(by='test type').max()[['u','v','r','delta','thrust']]

,u,v,r,delta,thrust
test type,,,,,
Circle,0.961770,-0.000000,-0.023974,-0.000000,10.394889
Circle + Drift,0.961770,0.167010,0.047949,-0.000000,9.709370
Circle + rudder angle,0.961770,-0.000000,-0.023974,0.069813,9.291126
Drift angle,0.961184,-0.033565,0.000000,-0.000000,10.664554
Rudder and drift angle,0.959427,-0.067090,0.000000,0.069813,9.410107
Rudder angle,0.961770,-0.000000,0.000000,-0.052360,9.229330
Rudder angle resistance (no propeller),0.961770,-0.000000,0.000000,-0.087266,0.000000
Thrust variation,0.961770,-0.000000,0.000000,-0.174533,13.721516
self propulsion,0.961770,-0.000000,0.000000,-0.000000,9.130484


In [15]:
df_VCT.groupby(by='test type').min()[['u','v','r','delta','thrust']]

,u,v,r,delta,thrust
test type,,,,,
Circle,0.961770,-0.000000,-0.127864,-0.000000,9.249258
Circle + Drift,0.947079,-0.167010,-0.047949,-0.000000,0.000000
Circle + rudder angle,0.961770,-0.000000,-0.023974,-0.069813,9.228015
Drift angle,0.903768,-0.328945,0.000000,-0.000000,9.124859
Rudder and drift angle,0.959427,-0.067090,0.000000,-0.069813,9.381886
Rudder angle,0.961770,-0.000000,0.000000,-0.261799,9.095363
Rudder angle resistance (no propeller),0.961770,-0.000000,0.000000,-0.261799,0.000000
Thrust variation,0.480885,-0.000000,0.000000,-0.174533,1.360484
self propulsion,0.480885,-0.000000,0.000000,-0.000000,2.577043


In [17]:
eq_N_D = model.expand_subsystemequations(model.N_D_eq)

In [18]:
eq_N_D.subs([
    (v,0),
    (r,0),
    (delta,0),
    (thrust_port,0),
    (thrust_stbd,0),
])

Eq(N_D, N_R*x_H + N_{0} + x_R*(D_R_port*sin(gamma_0_port + atan(V_R_y_port/V_R_x_C_port)) + L_R_port*cos(gamma_0_port + atan(V_R_y_port/V_R_x_C_port))) + x_R*(D_R_stbd*sin(gamma_0_stbd + atan(V_R_y_stbd/V_R_x_C_stbd)) + L_R_stbd*cos(gamma_0_stbd + atan(V_R_y_stbd/V_R_x_C_stbd))))

In [19]:
exclude_parameters

{'Y0': 0,
 'N0': 0,
 'Yvdeltadelta': 0,
 'Yvvdelta': 0,
 'Yrdeltadelta': 0,
 'Yrrdelta': 0,
 'Yvrdelta': 0,
 'Nrrdelta': 0,
 'Nvrdelta': 0,
 'Nvdeltadelta': 0,
 'Nrdeltadelta': 0,
 'Nvvdelta': 0,
 'Xthrustport': 0.883,
 'Xthruststbd': 0.883}

In [21]:
model_,models = regress_VCT(model=model, df_VCT=df_VCT, pipeline=pipeline, exclude_parameters=exclude_parameters)

[01/10/24 14:16:29] INFO     Fitting:rudder hull interaction aH                          ]8;id=319559;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/regression_pipeline.py\regression_pipeline.py]8;;\:]8;id=253012;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/regression_pipeline.py#297\297]8;;\

                    INFO     Fitting:rudder hull interaction xH                          ]8;id=462191;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/regression_pipeline.py\regression_pipeline.py]8;;\:]8;id=371152;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/regression_pipeline.py#297\297]8;;\

                    INFO     Fitting:resistance                                          ]8;id=975257;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/regression_pipeline.py\regression_pipeline.py]8;;\:]8;id=609527;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/regression_pipeline.py#297\297]8;;\

[01/10/24 14:16:30] INFO     Fitting:resistance fy                                       ]8;id=776424;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/regression_pipeline.py\regression_pipeline.py]8;;\:]8;id=750938;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/regression_pipeline.py#297\297]8;;\

skipping:resistance fy with equation: Eq(Y_H, Y_{0})


...

                    INFO                       Results: Ordinary least squares                         ]8;id=176350;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py\nodes.py]8;;\:]8;id=9982;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py#551\551]8;;\
                             ===================================================================                   
                             Model:              OLS              Adj. R-squared:     0.112                        
                             Dependent Variable: X_H              AIC:                -2139.8598                   
                             Date:               2024-01-10 14:16 BIC:                -2134.2518                   
                             No. Observations:   122              Log-Likelihood:     1071.9                       
                             Df Model:           1                F-statistic:        16.31                        
                             Df Residuals:       120              Prob (F-statistic): 9.53e-05                     
                             R-squared:          0.120            Scale:              1.3900e-09                   
                             ---------------------------------------------------------------------                 
                                          Coef.    Std.Err.      t      P>|t|     [0.025    0.975]                 
                             ---------------------------------------------------------------------                 
                             Xvr          0.0014     0.0003    4.0383   0.0001    0.0007    0.0021                 
                             const       -0.0000     0.0000   -5.0790   0.0000   -0.0000   -0.0000                 
                             -------------------------------------------------------------------                   
                             Omnibus:               72.204       Durbin-Watson:          1.113                     
                             Prob(Omnibus):         0.000        Jarque-Bera (JB):       264.557                   
                             Skew:                  -2.239       Prob(JB):               0.000                     
                             Kurtosis:              8.657        Condition No.:          102                       
                             ===================================================================                   
                                                                                                                   

                    INFO     Regression:Circle + Drift fy                                              ]8;id=669596;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py\nodes.py]8;;\:]8;id=450220;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py#549\549]8;;\

                    INFO                             Results: Ordinary least squares                   ]8;id=30597;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py\nodes.py]8;;\:]8;id=872046;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py#551\551]8;;\
                             =========================================================================             
                             =======                                                                               
                             Model:                  OLS              Adj. R-squared (uncentered):                 
                             0.477                                                                                 
                             Dependent Variable:     Y_H              AIC:                                         
                             -1927.1391                                                                            
                             Date:                   2024-01-10 14:16 BIC:                                         
                             -1921.5311                                                                            
                             No. Observations:       122              Log-Likelihood:                              
                             965.57                                                                                
                             Df Model:               2                F-statistic:                                 
                             56.53                                                                                 
                             Df Residuals:           120              Prob (F-statistic):                          
                             5.04e-18                                                                              
                             R-squared (uncentered): 0.485            Scale:                                       
                             7.9479e-09                                                                            
                             -------------------------------------------------------------------------             
                             ------------                                                                          
                                           Coef.       Std.Err.         t         P>|t|        [0.025              
                             0.975]                                                                                
                             -------------------------------------------------------------------------             
                             ------------                                                                          
                             Yvvr         -0.0925        0.0095      -9.7298      0.0000      -0.1113              
                             -0.0737                                                                               
                             Yvrr         -0.0436        0.0070      -6.1853      0.0000      -0.0576              
                             -0.0296                                                                               
                             -------------------------------------------------------------------------             
                             -------                                                                               
                             Omnibus:                    0.326             Durbin-Watson:                          
                             2.332                                                                                 
                             Prob(Omnibus):              0.850             Jarque-Bera (JB):                       
                             0.492                                                                         

                    INFO     Regression:Circle + Drift mz                                              ]8;id=724397;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py\nodes.py]8;;\:]8;id=390455;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py#549\549]8;;\

                    INFO                             Results: Ordinary least squares                   ]8;id=89949;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py\nodes.py]8;;\:]8;id=374205;file:///home/maa/dev/PHD/src/phd/pipelines/regression_VCT/nodes.py#551\551]8;;\
                             =========================================================================             
                             =======                                                                               
                             Model:                  OLS              Adj. R-squared (uncentered):                 
                             0.934                                                                                 
                             Dependent Variable:     N_H              AIC:                                         
                             -2463.9545                                                                            
                             Date:                   2024-01-10 14:16 BIC:                                         
                             -2458.3465                                                                            
                             No. Observations:       122              Log-Likelihood:                              
                             1234.0                                                                                
                             Df Model:               2                F-statistic:                                 
                             862.5                                                                                 
                             Df Residuals:           120              Prob (F-statistic):                          
                             6.18e-72                                                                              
                             R-squared (uncentered): 0.935            Scale:                                       
                             9.7567e-11                                                                            
                             -------------------------------------------------------------------------             
                             ------------                                                                          
                                          Coef.       Std.Err.         t          P>|t|        [0.025              
                             0.975]                                                                                
                             -------------------------------------------------------------------------             
                             ------------                                                                          
                             Nvvr        -0.0361        0.0011      -34.2485      0.0000      -0.0382              
                             -0.0340                                                                               
                             Nvrr        -0.0137        0.0008      -17.4895      0.0000      -0.0152              
                             -0.0121                                                                               
                             -------------------------------------------------------------------------             
                             -------                                                                               
                             Omnibus:                   0.014             Durbin-Watson:                           
                             2.075                                                                                 
                             Prob(Omnibus):             0.993             Jarque-Bera (JB):                        
                             0.026                                                                         

In [22]:
regression = models['Circle + Drift mz']

In [23]:
list(regression.params.keys())

['Nvvr', 'Nvrr']

In [24]:
parameters = pd.Series()
for name, regression in models.items():
    parameters[name] = list(regression.params.keys())

In [25]:
parameters

rudder hull interaction aH            [aH]
rudder hull interaction xH            [xH]
resistance                        [X0, Xu]
Drift angle fx                [Xvv, const]
Drift angle fy                  [Yv, Yvvv]
Drift angle mz                  [Nvvv, Nv]
Circle fx                     [Xrr, const]
Circle fy                       [Yrrr, Yr]
Circle mz                       [Nrrr, Nr]
Circle + Drift fx             [Xvr, const]
Circle + Drift fy             [Yvvr, Yvrr]
Circle + Drift mz             [Nvvr, Nvrr]
dtype: object

In [26]:
regression.eq

Eq(N_H, N_{0} + N_{rrr}*r**3 + N_{r}*r + N_{vrr}*r**2*v + N_{vvr}*r*v**2 + N_{vvv}*v**3 + N_{v}*v)

In [27]:
eq_latex = sp.latex(regression.eq.lhs)
eq_latex_prime = f"{{{eq_latex}}}'"
eq_latex_prime

"{N_{H}}'"

In [28]:
def to_latex_prime(expression):
    eq_latex = sp.latex(expression)
    eq_latex_prime = f"${{{eq_latex}}}'$"
    eq_latex_prime = eq_latex_prime.replace('delta',r'\delta')
    return eq_latex_prime

In [33]:
name_replace={
    "rudder hull interaction aH":"Rudder angle",
    "rudder hull interaction xH":"Rudder angle",
    "resistance":"Self propulsion",
}

In [37]:
table_regression_pipeline = pd.DataFrame()
found = set([u,v,r,delta,thrust_port,thrust_stbd,p.Xthruststbd,p.Xthrustport,y_p_port, y_p_stbd,p.Y0,p.N0,Y_R,N_R])
for name, regression in models.items():
    new_parameters = regression.eq.rhs.free_symbols - found
    found=found.union(regression.eq.rhs.free_symbols)
    
    test_type = str(name)
    test_type = test_type.replace(" fx","")
    test_type = test_type.replace(" fy","")
    test_type = test_type.replace(" mz","")
    table_regression_pipeline.loc[name,'Test type'] = name_replace.get(test_type,test_type) 
    table_regression_pipeline.loc[name,'Label'] = to_latex_prime(regression.eq.lhs)
    table_regression_pipeline.loc[name,'Features'] = " ".join([to_latex_prime(parameter) for parameter in new_parameters])

In [38]:
table_regression_pipeline

,Test type,Label,Features
rudder hull interaction aH,Rudder angle,${Y_{D}}'$,${a_{H}}'$
rudder hull interaction xH,Rudder angle,${N_{D}}'$,${x_{H}}'$
resistance,Self propulsion,${X_{H}}'$,${X_{u}}'$ ${X_{0}}'$
Drift angle fx,Drift angle,${X_{H}}'$,${X_{vv}}'$
Drift angle fy,Drift angle,${Y_{H}}'$,${Y_{vvv}}'$ ${Y_{v}}'$
Drift angle mz,Drift angle,${N_{H}}'$,${N_{v}}'$ ${N_{vvv}}'$
Circle fx,Circle,${X_{H}}'$,${X_{rr}}'$
Circle fy,Circle,${Y_{H}}'$,${Y_{rrr}}'$ ${Y_{r}}'$
Circle mz,Circle,${N_{H}}'$,${N_{rrr}}'$ ${N_{r}}'$
Circle + Drift fx,Circle + Drift,${X_{H}}'$,${X_{vr}}'$


In [39]:
table_regression_pipeline.to_csv(paper.file_path_with_nb_ref("regression_pipeline_semiempirical.csv", directory='tables'), index=False)